In [107]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
  .appName('DADOS PUBLICOS CSV') \
  .getOrCreate()

In [120]:
from pyspark.sql.functions import row_number, monotonically_increasing_id
from pyspark.sql import Window

In [108]:
spark.conf.set("spark.sql.repl.eagerEval.enabled",True)

In [109]:
!hdfs dfs -ls 'gs://neo4j_nunes/dp_socios_dataset/2_CSV_DATASET'

Found 37 items
-rwx------   3 root root          0 2022-06-15 13:49 gs://neo4j_nunes/dp_socios_dataset/2_CSV_DATASET/ATRIBUTO_MUNICIPIO
-rwx------   3 root root          0 2022-06-15 13:49 gs://neo4j_nunes/dp_socios_dataset/2_CSV_DATASET/ATRIBUTO_PAIS
-rwx------   3 root root  705403793 2022-06-15 13:51 gs://neo4j_nunes/dp_socios_dataset/2_CSV_DATASET/EMPRESASCSV01
-rwx------   3 root root  325481460 2022-06-15 13:50 gs://neo4j_nunes/dp_socios_dataset/2_CSV_DATASET/EMPRESASCSV02
-rwx------   3 root root  343862867 2022-06-15 13:49 gs://neo4j_nunes/dp_socios_dataset/2_CSV_DATASET/EMPRESASCSV03
-rwx------   3 root root  349094408 2022-06-15 13:51 gs://neo4j_nunes/dp_socios_dataset/2_CSV_DATASET/EMPRESASCSV04
-rwx------   3 root root  354018482 2022-06-15 13:50 gs://neo4j_nunes/dp_socios_dataset/2_CSV_DATASET/EMPRESASCSV05
-rwx------   3 root root  352921019 2022-06-15 13:50 gs://neo4j_nunes/dp_socios_dataset/2_CSV_DATASET/EMPRESASCSV06
-rwx------   3 root root  358711865 2022-06-15 13:49

In [110]:
df_1 = spark \
  .read \
  .option ( "inferSchema" , "false" ) \
  .option ( "header" , "false" ) \
  .option("sep", ";") \
  .csv ( "gs://neo4j_nunes/dp_socios_dataset/2_CSV_DATASET/EMPRESASCSV*" )

df_1.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)



In [111]:
df_1.tail(25)

[Row(_c0='98768211', _c1='FARMACIA CONCEICAO LTDA', _c2='2062', _c3='49', _c4='0,00', _c5='05', _c6=None),
 Row(_c0='98817851', _c1='CARLOS BARRETO', _c2='2135', _c3='50', _c4='0,00', _c5='05', _c6=None),
 Row(_c0='98817992', _c1='ABELARDO LUIZ CRISTIANO', _c2='4014', _c3='34', _c4='0,00', _c5='05', _c6=None),
 Row(_c0='98818198', _c1='MAURICIO DE LIMA JAQUES', _c2='2135', _c3='50', _c4='0,00', _c5='05', _c6=None),
 Row(_c0='98818214', _c1='ACAO SOCIAL PAROQUIAL NOSSA SENHORA DA SAUDE', _c2='3999', _c3='16', _c4='0,00', _c5='05', _c6=None),
 Row(_c0='98818271', _c1='LUIZ DE OLIVEIRA', _c2='2135', _c3='50', _c4='0,00', _c5='05', _c6=None),
 Row(_c0='98818370', _c1='ADAO DIRCY DIAS', _c2='2135', _c3='50', _c4='0,00', _c5='05', _c6=None),
 Row(_c0='98818487', _c1='ASSIS ROSSATO', _c2='2135', _c3='50', _c4='0,00', _c5='05', _c6=None),
 Row(_c0='98818529', _c1='JOSE CANDIDO DE OLIVEIRA', _c2='2135', _c3='50', _c4='0,00', _c5='05', _c6=None),
 Row(_c0='98818834', _c1='AGENCIA SOLYMAR LTDA', 

In [112]:
df2 = df_1.selectExpr("_c0 as CNPJ_BASICO","_c1 as NOME_EMPRESARIAL","_c2 as NATUREZA_JURIDICA","_c3 as QUALIFICACAO_RESPONSAVEL","_c4 as CAPITAL_SOCIAL","_c5 as PORTE_EMPRESA","_c6 as ENTE_FEDERATIVO")

In [113]:
df2.show(5)

+-----------+--------------------+-----------------+------------------------+--------------+-------------+---------------+
|CNPJ_BASICO|    NOME_EMPRESARIAL|NATUREZA_JURIDICA|QUALIFICACAO_RESPONSAVEL|CAPITAL_SOCIAL|PORTE_EMPRESA|ENTE_FEDERATIVO|
+-----------+--------------------+-----------------+------------------------+--------------+-------------+---------------+
|   41273602|FABIO SOUZA DO RO...|             2135|                      50|      15000,00|           01|           null|
|   41273603|GRAFLINE ACESSORI...|             2062|                      49|      10000,00|           01|           null|
|   41273604|RUMO - ESTUDIO DE...|             2062|                      49|      10000,00|           01|           null|
|   41273605|WALLACE DE OLIVEI...|             2135|                      50|       1000,00|           01|           null|
|   41273606|MARCOS CESAR DE M...|             2135|                      50|      72000,00|           01|           null|
+-----------+---

In [115]:
df_empresas = df2.withColumn("EMPRESA_ID",row_number().over(Window.orderBy(monotonically_increasing_id()))-1)

In [116]:
df_empresas.show(5)

22/06/15 13:58:09 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/15 13:58:09 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/15 13:58:59 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/15 13:58:59 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-----------+--------------------+-----------------+------------------------+--------------+-------------+---------------+----------+
|CNPJ_BASICO|    NOME_EMPRESARIAL|NATUREZA_JURIDICA|QUALIFICACAO_RESPONSAVEL|CAPITAL_SOCIAL|PORTE_EMPRESA|ENTE_FEDERATIVO|EMPRESA_ID|
+-----------+--------------------+-----------------+------------------------+--------------+-------------+---------------+----------+
|   41273602|FABIO SOUZA DO RO...|             2135|                      50|      15000,00|           01|           null|         0|
|   41273603|GRAFLINE ACESSORI...|             2062|                      49|      10000,00|           01|           null|         1|
|   41273604|RUMO - ESTUDIO DE...|             2062|                      49|      10000,00|           01|           null|         2|
|   41273605|WALLACE DE OLIVEI...|             2135|                      50|       1000,00|           01|           null|         3|
|   41273606|MARCOS CESAR DE M...|             2135|          

In [117]:
df_empresas.coalesce(1).write \
  .mode('overwrite') \
  .option("header", "True") \
  .csv("gs://neo4j_nunes/dp_socios_dataset/2_CSV_DATASET/ALL_EMPRESAS")

22/06/15 14:00:14 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/15 14:00:14 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/15 14:01:00 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/15 14:01:00 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [118]:
df_1 = spark \
  .read \
  .option ( "inferSchema" , "false" ) \
  .option ( "header" , "false" ) \
  .option("sep", ";") \
  .csv ( "gs://neo4j_nunes/dp_socios_dataset/2_CSV_DATASET/ESTABELECIMENTOSCSV*" )

df_1.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)
 |-- _c11: string (nullable = true)
 |-- _c12: string (nullable = true)
 |-- _c13: string (nullable = true)
 |-- _c14: string (nullable = true)
 |-- _c15: string (nullable = true)
 |-- _c16: string (nullable = true)
 |-- _c17: string (nullable = true)
 |-- _c18: string (nullable = true)
 |-- _c19: string (nullable = true)
 |-- _c20: string (nullable = true)
 |-- _c21: string (nullable = true)
 |-- _c22: string (nullable = true)
 |-- _c23: string (nullable = true)
 |-- _c24: string (nullable = true)
 |-- _c25: string (nullable = true)
 |-- _c26: string (nullable = true)
 |-- _c27: string (nullable = tru

In [122]:
df_1.show(5)

+--------+----+---+---+---------------+---+--------+---+----+----+--------+-------+--------------------+-------+--------------------+----+------------------+---------------+--------+----+----+----+--------+----+----+----+--------+----+----+----+
|     _c0| _c1|_c2|_c3|            _c4|_c5|     _c6|_c7| _c8| _c9|    _c10|   _c11|                _c12|   _c13|                _c14|_c15|              _c16|           _c17|    _c18|_c19|_c20|_c21|    _c22|_c23|_c24|_c25|    _c26|_c27|_c28|_c29|
+--------+----+---+---+---------------+---+--------+---+----+----+--------+-------+--------------------+-------+--------------------+----+------------------+---------------+--------+----+----+----+--------+----+----+----+--------+----+----+----+
|07100686|0001| 31|  1|   CAFE SOBERBO| 08|20200803| 01|null|null|20040817|1081302|             7739099|ESTRADA|CACONDE/MUZAMBINH...| S/N|              null|       SAO JOAO|13770000|  SP|6275|  19|36621414|null|null|  19|36621414|null|null|null|
|41079476|0001| 

In [126]:
df2 = df_1.selectExpr("_c0 as CNPJ_BASICO","_c1 as CNPJ_ORDEM","_c2 as CNPJ_DV","_c3 as CODIGO_IDENTIFICADOR","_c4 as NOME_FANTASIA","_c5 as SITUACAO_CADASTRAL","_c6 as DATA_CADASTRAL","_c7 as MOTIVO_CADASTRAL","_c8 as NOME_CIDADE","_c9 as CODIGO_PAIS","_c10 as DATA_INICIO","_c11 as CNAE_PRINCIPAL","_c12 as CNAE_SECUNDARIO","_c13 as TIPO_LOGRADOURO","_c14 as NOME_LOGRADOURO","_c15 as NUMERO_ESTABELECIMENTO","_c16 as COMPLEMENTO_ESTABELECIMENTO","_c17 as BAIRRO_ESTABELECIMENTO","_c18 as CEP_ESTABELECIMENTO","_c19 as UF_ESTABELECIMENTO","_c20 as MUNICIPIO_ESTABELECIMENTO","_c21 as DDD_UM","_c22 as TELEFONE_UM","_c23 as DDD_DOIS","_c24 as TELEFONE_DOIS","_c25 as DDD_FAX","_c26 as TELEFONE_FAX","_c27 as EMAIL","_c28 as SITUACAO_ESPECIAL","_c29 as DATA_ESPECIAL")

In [127]:
df2.show(5)

+-----------+----------+-------+--------------------+---------------+------------------+--------------+----------------+-----------+-----------+-----------+--------------+--------------------+---------------+--------------------+----------------------+---------------------------+----------------------+-------------------+------------------+-------------------------+------+-----------+--------+-------------+-------+------------+-----+-----------------+-------------+
|CNPJ_BASICO|CNPJ_ORDEM|CNPJ_DV|CODIGO_IDENTIFICADOR|  NOME_FANTASIA|SITUACAO_CADASTRAL|DATA_CADASTRAL|MOTIVO_CADASTRAL|NOME_CIDADE|CODIGO_PAIS|DATA_INICIO|CNAE_PRINCIPAL|     CNAE_SECUNDARIO|TIPO_LOGRADOURO|     NOME_LOGRADOURO|NUMERO_ESTABELECIMENTO|COMPLEMENTO_ESTABELECIMENTO|BAIRRO_ESTABELECIMENTO|CEP_ESTABELECIMENTO|UF_ESTABELECIMENTO|MUNICIPIO_ESTABELECIMENTO|DDD_UM|TELEFONE_UM|DDD_DOIS|TELEFONE_DOIS|DDD_FAX|TELEFONE_FAX|EMAIL|SITUACAO_ESPECIAL|DATA_ESPECIAL|
+-----------+----------+-------+--------------------+-------

In [128]:
df_empresas = df2.withColumn("ESTABELECIMENTO_ID",row_number().over(Window.orderBy(monotonically_increasing_id()))-1)

In [ ]:
df_empresas.show(5)

22/06/15 14:26:39 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/15 14:26:39 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/15 14:28:45 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/15 14:28:45 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-----------+----------+-------+--------------------+---------------+------------------+--------------+----------------+-----------+-----------+-----------+--------------+--------------------+---------------+--------------------+----------------------+---------------------------+----------------------+-------------------+------------------+-------------------------+------+-----------+--------+-------------+-------+------------+-----+-----------------+-------------+------------------+
|CNPJ_BASICO|CNPJ_ORDEM|CNPJ_DV|CODIGO_IDENTIFICADOR|  NOME_FANTASIA|SITUACAO_CADASTRAL|DATA_CADASTRAL|MOTIVO_CADASTRAL|NOME_CIDADE|CODIGO_PAIS|DATA_INICIO|CNAE_PRINCIPAL|     CNAE_SECUNDARIO|TIPO_LOGRADOURO|     NOME_LOGRADOURO|NUMERO_ESTABELECIMENTO|COMPLEMENTO_ESTABELECIMENTO|BAIRRO_ESTABELECIMENTO|CEP_ESTABELECIMENTO|UF_ESTABELECIMENTO|MUNICIPIO_ESTABELECIMENTO|DDD_UM|TELEFONE_UM|DDD_DOIS|TELEFONE_DOIS|DDD_FAX|TELEFONE_FAX|EMAIL|SITUACAO_ESPECIAL|DATA_ESPECIAL|ESTABELECIMENTO_ID|
+-----------+---------

In [130]:
df_empresas.coalesce(1).write \
  .mode('overwrite') \
  .option("header", "True") \
  .csv("gs://neo4j_nunes/dp_socios_dataset/2_CSV_DATASET/ALL_ESTABELECIMENTOS")

22/06/15 14:35:21 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/15 14:35:21 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/15 14:37:21 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/15 14:37:21 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [131]:
df_1 = spark \
  .read \
  .option ( "inferSchema" , "false" ) \
  .option ( "header" , "false" ) \
  .option("sep", ";") \
  .csv ( "gs://neo4j_nunes/dp_socios_dataset/2_CSV_DATASET/SOCIOSCSV*" )

df_1.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)



In [132]:
df_1.show(5)

+--------+---+--------------------+-----------+---+--------+----+-----------+----+---+----+
|     _c0|_c1|                 _c2|        _c3|_c4|     _c5| _c6|        _c7| _c8|_c9|_c10|
+--------+---+--------------------+-----------+---+--------+----+-----------+----+---+----+
|02954739|  2|LIDIANE BORGES PI...|***852188**| 49|19990129|null|***000000**|null| 00|   5|
|06888391|  2|WAGNER APARECIDO ...|***345608**| 49|20040630|null|***000000**|null| 00|   5|
|06888430|  2|SERGIO RICARDO LI...|***542024**| 16|20070903|null|***000000**|null| 00|   4|
|02522368|  2|MANOEL ANTONIO SI...|***363378**| 22|19980513|null|***000000**|null| 00|   8|
|02522368|  2|MIRTES MARIA DASC...|***168448**| 49|19980513|null|***000000**|null| 00|   8|
+--------+---+--------------------+-----------+---+--------+----+-----------+----+---+----+
only showing top 5 rows



In [133]:
df2 = df_1.selectExpr("_c0 as CNPJ_BASICO","_c1 as IDENTIFICADOR_SOCIO","_c2 as NOME_SOCIO","_c3 as CX_SOCIO","_c4 as QUALIFICACAO_SOCIO","_c5 as DATA_SOCIEDADE","_c6 as PAIS","_c7 as NUMERO_REPRESENTANTE","_c8 as NOME_REPRESENTANTE","_c9 as QUALIFICACAO_REPRESENTANTE","_c10 as FAIXA_ETARIA")

In [134]:
df2.show(5)

+-----------+-------------------+--------------------+-----------+------------------+--------------+----+--------------------+------------------+--------------------------+------------+
|CNPJ_BASICO|IDENTIFICADOR_SOCIO|          NOME_SOCIO|   CX_SOCIO|QUALIFICACAO_SOCIO|DATA_SOCIEDADE|PAIS|NUMERO_REPRESENTANTE|NOME_REPRESENTANTE|QUALIFICACAO_REPRESENTANTE|FAIXA_ETARIA|
+-----------+-------------------+--------------------+-----------+------------------+--------------+----+--------------------+------------------+--------------------------+------------+
|   02954739|                  2|LIDIANE BORGES PI...|***852188**|                49|      19990129|null|         ***000000**|              null|                        00|           5|
|   06888391|                  2|WAGNER APARECIDO ...|***345608**|                49|      20040630|null|         ***000000**|              null|                        00|           5|
|   06888430|                  2|SERGIO RICARDO LI...|***542024**|    

In [135]:
df_empresas = df2.withColumn("SOCIO_ID",row_number().over(Window.orderBy(monotonically_increasing_id()))-1)

In [136]:
df_empresas.show(5)

22/06/15 14:56:54 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/15 14:56:54 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/15 14:57:27 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/15 14:57:27 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-----------+-------------------+--------------------+-----------+------------------+--------------+----+--------------------+------------------+--------------------------+------------+--------+
|CNPJ_BASICO|IDENTIFICADOR_SOCIO|          NOME_SOCIO|   CX_SOCIO|QUALIFICACAO_SOCIO|DATA_SOCIEDADE|PAIS|NUMERO_REPRESENTANTE|NOME_REPRESENTANTE|QUALIFICACAO_REPRESENTANTE|FAIXA_ETARIA|SOCIO_ID|
+-----------+-------------------+--------------------+-----------+------------------+--------------+----+--------------------+------------------+--------------------------+------------+--------+
|   02954739|                  2|LIDIANE BORGES PI...|***852188**|                49|      19990129|null|         ***000000**|              null|                        00|           5|       0|
|   06888391|                  2|WAGNER APARECIDO ...|***345608**|                49|      20040630|null|         ***000000**|              null|                        00|           5|       1|
|   06888430|            

In [137]:
df_empresas.coalesce(1).write \
  .mode('overwrite') \
  .option("header", "True") \
  .csv("gs://neo4j_nunes/dp_socios_dataset/2_CSV_DATASET/ALL_SOCIOS")

22/06/15 14:58:53 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/15 14:58:53 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/15 14:59:26 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/15 14:59:26 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [151]:
df_1 = spark \
  .read \
  .option ( "inferSchema" , "false" ) \
  .option ( "header" , "false" ) \
  .option("sep", ";") \
  .csv ( "gs://neo4j_nunes/dp_socios_dataset/2_CSV_DATASET/ATRIBUTO_MUNICIPIO*" )

In [152]:
df_1.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)



In [153]:
df_1.show(5)

+----+--------------------+
| _c0|                 _c1|
+----+--------------------+
|0001|       GUAJARA-MIRIM|
|0002|ALTO ALEGRE DOS P...|
|0003|         PORTO VELHO|
|0004|             BURITIS|
|0005|           JI-PARANA|
+----+--------------------+
only showing top 5 rows



In [154]:
df2 = df_1.selectExpr("_c0 as CODIGO_MUNICIPIO","_c1 as NOME_MUNICIPIO")

In [155]:
df2.show(5)

+----------------+--------------------+
|CODIGO_MUNICIPIO|      NOME_MUNICIPIO|
+----------------+--------------------+
|            0001|       GUAJARA-MIRIM|
|            0002|ALTO ALEGRE DOS P...|
|            0003|         PORTO VELHO|
|            0004|             BURITIS|
|            0005|           JI-PARANA|
+----------------+--------------------+
only showing top 5 rows



In [156]:
df_empresas = df2.withColumn("MUNICIPIO_ID",row_number().over(Window.orderBy(monotonically_increasing_id()))-1)

In [157]:
df_empresas.show(5)

22/06/15 15:22:43 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/15 15:22:43 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+----------------+--------------------+------------+
|CODIGO_MUNICIPIO|      NOME_MUNICIPIO|MUNICIPIO_ID|
+----------------+--------------------+------------+
|            0001|       GUAJARA-MIRIM|           0|
|            0002|ALTO ALEGRE DOS P...|           1|
|            0003|         PORTO VELHO|           2|
|            0004|             BURITIS|           3|
|            0005|           JI-PARANA|           4|
+----------------+--------------------+------------+
only showing top 5 rows



22/06/15 15:22:43 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/15 15:22:43 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [158]:
df_empresas.coalesce(1).write \
  .mode('overwrite') \
  .option("header", "True") \
  .csv("gs://neo4j_nunes/dp_socios_dataset/2_CSV_DATASET/ALL_ATRIBUTO_MUNICIPIO")

22/06/15 15:22:52 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/15 15:22:52 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/15 15:22:53 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/15 15:22:53 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [159]:
df_1 = spark \
  .read \
  .option ( "inferSchema" , "false" ) \
  .option ( "header" , "false" ) \
  .option("sep", ";") \
  .csv ( "gs://neo4j_nunes/dp_socios_dataset/2_CSV_DATASET/ATRIBUTO_CNAE" )

In [161]:
df_1.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)



In [162]:
df_1.show(5)

+-------+--------------------+
|    _c0|                 _c1|
+-------+--------------------+
|0111301|    Cultivo de arroz|
|0111302|    Cultivo de milho|
|0111303|    Cultivo de trigo|
|0111399|Cultivo de outros...|
|0112101|Cultivo de algod�...|
+-------+--------------------+
only showing top 5 rows



In [163]:
df2 = df_1.selectExpr("_c0 as CODIGO_CNAE","_c1 as ATIVIDADE_ECONOMICA")

In [164]:
df2.show(5)

+-----------+--------------------+
|CODIGO_CNAE| ATIVIDADE_ECONOMICA|
+-----------+--------------------+
|    0111301|    Cultivo de arroz|
|    0111302|    Cultivo de milho|
|    0111303|    Cultivo de trigo|
|    0111399|Cultivo de outros...|
|    0112101|Cultivo de algod�...|
+-----------+--------------------+
only showing top 5 rows



In [165]:
df_empresas = df2.withColumn("CNAE_ID",row_number().over(Window.orderBy(monotonically_increasing_id()))-1)

In [166]:
df_empresas.show(5)

22/06/16 02:25:59 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/16 02:25:59 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-----------+--------------------+-------+
|CODIGO_CNAE| ATIVIDADE_ECONOMICA|CNAE_ID|
+-----------+--------------------+-------+
|    0111301|    Cultivo de arroz|      0|
|    0111302|    Cultivo de milho|      1|
|    0111303|    Cultivo de trigo|      2|
|    0111399|Cultivo de outros...|      3|
|    0112101|Cultivo de algod�...|      4|
+-----------+--------------------+-------+
only showing top 5 rows



22/06/16 02:25:59 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/16 02:25:59 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [167]:
df_empresas.coalesce(1).write \
  .mode('overwrite') \
  .option("header", "True") \
  .csv("gs://neo4j_nunes/dp_socios_dataset/2_CSV_DATASET/ALL_ATRIBUTO_CNAE")

22/06/16 02:26:12 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/16 02:26:12 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/16 02:26:13 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/16 02:26:13 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [168]:
df_1 = spark \
  .read \
  .option ( "inferSchema" , "false" ) \
  .option ( "header" , "false" ) \
  .option("sep", ";") \
  .csv ( "gs://neo4j_nunes/dp_socios_dataset/2_CSV_DATASET/ATRIBUTO_PAIS" )

In [169]:
df_1.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)



In [170]:
df_1.show(5)

+---+--------------------+
|_c0|                 _c1|
+---+--------------------+
|000|       COLIS POSTAUX|
|013|         AFEGANISTAO|
|017|             ALBANIA|
|020|ALBORAN-PEREJIL,I...|
|023|            ALEMANHA|
+---+--------------------+
only showing top 5 rows



In [171]:
df2 = df_1.selectExpr("_c0 as CODIGO_PAIS","_c1 as NOME_PAIS")

In [172]:
df2.show(5)

+-----------+--------------------+
|CODIGO_PAIS|           NOME_PAIS|
+-----------+--------------------+
|        000|       COLIS POSTAUX|
|        013|         AFEGANISTAO|
|        017|             ALBANIA|
|        020|ALBORAN-PEREJIL,I...|
|        023|            ALEMANHA|
+-----------+--------------------+
only showing top 5 rows



In [173]:
df_empresas = df2.withColumn("PAIS_ID",row_number().over(Window.orderBy(monotonically_increasing_id()))-1)

In [174]:
df_empresas.show(5)

22/06/16 02:29:16 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/16 02:29:16 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-----------+--------------------+-------+
|CODIGO_PAIS|           NOME_PAIS|PAIS_ID|
+-----------+--------------------+-------+
|        000|       COLIS POSTAUX|      0|
|        013|         AFEGANISTAO|      1|
|        017|             ALBANIA|      2|
|        020|ALBORAN-PEREJIL,I...|      3|
|        023|            ALEMANHA|      4|
+-----------+--------------------+-------+
only showing top 5 rows



22/06/16 02:29:16 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/16 02:29:16 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [175]:
df_empresas.coalesce(1).write \
  .mode('overwrite') \
  .option("header", "True") \
  .csv("gs://neo4j_nunes/dp_socios_dataset/2_CSV_DATASET/ALL_ATRIBUTO_PAIS")

22/06/16 02:29:20 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/16 02:29:20 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/16 02:29:21 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/16 02:29:21 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [176]:
df_1 = spark \
  .read \
  .option ( "inferSchema" , "false" ) \
  .option ( "header" , "false" ) \
  .option("sep", ";") \
  .csv ( "gs://neo4j_nunes/dp_socios_dataset/2_CSV_DATASET/NATUREZA_JURIDICA" )

In [177]:
df_1.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)



In [178]:
df_1.show(5)

+----+--------------------+
| _c0|                 _c1|
+----+--------------------+
|0000|Natureza Jur�dica...|
|3271|�rg�o de Dire��o ...|
|3280|Comit� Financeiro...|
|3298|Frente Plebiscit�...|
|3301|Organiza��o Socia...|
+----+--------------------+
only showing top 5 rows



In [179]:
df2 = df_1.selectExpr("_c0 as CODIGO_NATUREZA_JURIDICA","_c1 as NOME_NATUREZA_JURIDICA")

In [180]:
df2.show(5)

+------------------------+----------------------+
|CODIGO_NATUREZA_JURIDICA|NOME_NATUREZA_JURIDICA|
+------------------------+----------------------+
|                    0000|  Natureza Jur�dica...|
|                    3271|  �rg�o de Dire��o ...|
|                    3280|  Comit� Financeiro...|
|                    3298|  Frente Plebiscit�...|
|                    3301|  Organiza��o Socia...|
+------------------------+----------------------+
only showing top 5 rows



In [181]:
df_empresas = df2.withColumn("NATUREZA_JURIDICA_ID",row_number().over(Window.orderBy(monotonically_increasing_id()))-1)

In [182]:
df_empresas.show(5)

22/06/16 02:32:03 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/16 02:32:03 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+------------------------+----------------------+--------------------+
|CODIGO_NATUREZA_JURIDICA|NOME_NATUREZA_JURIDICA|NATUREZA_JURIDICA_ID|
+------------------------+----------------------+--------------------+
|                    0000|  Natureza Jur�dica...|                   0|
|                    3271|  �rg�o de Dire��o ...|                   1|
|                    3280|  Comit� Financeiro...|                   2|
|                    3298|  Frente Plebiscit�...|                   3|
|                    3301|  Organiza��o Socia...|                   4|
+------------------------+----------------------+--------------------+
only showing top 5 rows



22/06/16 02:32:03 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/16 02:32:03 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [183]:
df_empresas.coalesce(1).write \
  .mode('overwrite') \
  .option("header", "True") \
  .csv("gs://neo4j_nunes/dp_socios_dataset/2_CSV_DATASET/ALL_NATUREZA_JURIDICA")

22/06/16 02:32:07 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/16 02:32:07 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/16 02:32:08 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/16 02:32:08 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [184]:
df_1 = spark \
  .read \
  .option ( "inferSchema" , "false" ) \
  .option ( "header" , "false" ) \
  .option("sep", ";") \
  .csv ( "gs://neo4j_nunes/dp_socios_dataset/2_CSV_DATASET/QUALIFICACAO_SOCIOS" )

In [185]:
df_1.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)



In [186]:
df_1.show(5)

+---+--------------------+
|_c0|                 _c1|
+---+--------------------+
| 00|       N�o informada|
| 05|       Administrador|
| 08|Conselheiro de Ad...|
| 09|             Curador|
| 10|             Diretor|
+---+--------------------+
only showing top 5 rows



In [187]:
df2 = df_1.selectExpr("_c0 as CODIGO_QUALIFICACAO_SOCIOS","_c1 as NOME_QUALIFICACAO_SOCIOS")

In [188]:
df2.show(5)

+--------------------------+------------------------+
|CODIGO_QUALIFICACAO_SOCIOS|NOME_QUALIFICACAO_SOCIOS|
+--------------------------+------------------------+
|                        00|           N�o informada|
|                        05|           Administrador|
|                        08|    Conselheiro de Ad...|
|                        09|                 Curador|
|                        10|                 Diretor|
+--------------------------+------------------------+
only showing top 5 rows



In [189]:
df_empresas = df2.withColumn("QUALIFICACAO_SOCIOS_ID",row_number().over(Window.orderBy(monotonically_increasing_id()))-1)

In [190]:
df_empresas.show(5)

22/06/16 02:34:56 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/16 02:34:56 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+--------------------------+------------------------+----------------------+
|CODIGO_QUALIFICACAO_SOCIOS|NOME_QUALIFICACAO_SOCIOS|QUALIFICACAO_SOCIOS_ID|
+--------------------------+------------------------+----------------------+
|                        00|           N�o informada|                     0|
|                        05|           Administrador|                     1|
|                        08|    Conselheiro de Ad...|                     2|
|                        09|                 Curador|                     3|
|                        10|                 Diretor|                     4|
+--------------------------+------------------------+----------------------+
only showing top 5 rows



22/06/16 02:34:57 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/16 02:34:57 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [191]:
df_empresas.coalesce(1).write \
  .mode('overwrite') \
  .option("header", "True") \
  .csv("gs://neo4j_nunes/dp_socios_dataset/2_CSV_DATASET/ALL_QUALIFICACAO_SOCIOS")

22/06/16 02:35:01 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/16 02:35:01 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/16 02:35:02 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/16 02:35:02 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [198]:
df_1 = spark \
  .read \
  .option ( "inferSchema" , "false" ) \
  .option ( "header" , "true" ) \
  .option("sep", ",") \
  .csv ( "gs://neo4j_nunes/dp_socios_dataset/2_CSV_DATASET/REGIMES_TRIBUTARIOS" )

In [199]:
df_1.printSchema()

root
 |-- ano: string (nullable = true)
 |-- cnpj: string (nullable = true)
 |-- forma_de_tributacao: string (nullable = true)
 |-- municipio: string (nullable = true)
 |-- uf: string (nullable = true)



In [200]:
df_1.show(5)

+----+------------------+-------------------+--------------------+---+
| ano|              cnpj|forma_de_tributacao|           municipio| uf|
+----+------------------+-------------------+--------------------+---+
|2020|00.000.049/0001-44|     ISENTA DO IRPJ|           SAO PAULO| SP|
|2020|00.000.067/0001-26|      IMUNE DO IRPJ|SANTA HELENA DE G...| GO|
|2020|00.000.104/0001-04|      IMUNE DO IRPJ|           SAO PAULO| SP|
|2020|00.000.132/0001-13|     ISENTA DO IRPJ|           SAO PAULO| SP|
|2020|00.000.138/0001-90|      IMUNE DO IRPJ|           SAO PAULO| SP|
+----+------------------+-------------------+--------------------+---+
only showing top 5 rows



In [ ]:
#df2 = df_1.selectExpr("_c0 as CODIGO_MUNICIPIO","_c1 as NOME_MUNICIPIO")

In [ ]:
#df2.show(5)

In [201]:
df_empresas = df2.withColumn("REGIMES_TRIBUTARIOS_ID",row_number().over(Window.orderBy(monotonically_increasing_id()))-1)

In [202]:
df_empresas.show(5)

22/06/16 02:38:41 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/16 02:38:41 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+--------------------------+------------------------+----------------------+
|CODIGO_QUALIFICACAO_SOCIOS|NOME_QUALIFICACAO_SOCIOS|REGIMES_TRIBUTARIOS_ID|
+--------------------------+------------------------+----------------------+
|                        00|           N�o informada|                     0|
|                        05|           Administrador|                     1|
|                        08|    Conselheiro de Ad...|                     2|
|                        09|                 Curador|                     3|
|                        10|                 Diretor|                     4|
+--------------------------+------------------------+----------------------+
only showing top 5 rows



22/06/16 02:38:42 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/16 02:38:42 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [203]:
df_empresas.coalesce(1).write \
  .mode('overwrite') \
  .option("header", "True") \
  .csv("gs://neo4j_nunes/dp_socios_dataset/2_CSV_DATASET/ALL_REGIMES_TRIBUTARIOS")

22/06/16 02:39:05 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/16 02:39:05 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/16 02:39:06 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/16 02:39:06 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [204]:
df_1 = spark \
  .read \
  .option ( "inferSchema" , "false" ) \
  .option ( "header" , "false" ) \
  .option("sep", ";") \
  .csv ( "gs://neo4j_nunes/dp_socios_dataset/2_CSV_DATASET/SIMPLES_MEI" )

In [205]:
df_1.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)



In [206]:
df_1.show(5)

+--------+---+--------+--------+---+--------+--------+
|     _c0|_c1|     _c2|     _c3|_c4|     _c5|     _c6|
+--------+---+--------+--------+---+--------+--------+
|00000000|  N|20070701|20070701|  N|20090701|20090701|
|00000006|  N|20180101|20191231|  N|00000000|00000000|
|00000008|  N|20140101|20211231|  N|00000000|00000000|
|00000011|  S|20070701|00000000|  N|00000000|00000000|
|00000013|  S|20090101|00000000|  N|00000000|00000000|
+--------+---+--------+--------+---+--------+--------+
only showing top 5 rows



In [207]:
df2 = df_1.selectExpr("_c0 as CNPJ_BASICO","_c1 as OPCAO_SIMPLES","_c2 as DATA_SIMPLES","_c3 as SIMPLES_EXCLUSAO","_c4 as OPCAO_MEI","_c5 as DATA_MEI","_c6 as MEI_EXCLUSAO")

In [208]:
df2.show(5)

+-----------+-------------+------------+----------------+---------+--------+------------+
|CNPJ_BASICO|OPCAO_SIMPLES|DATA_SIMPLES|SIMPLES_EXCLUSAO|OPCAO_MEI|DATA_MEI|MEI_EXCLUSAO|
+-----------+-------------+------------+----------------+---------+--------+------------+
|   00000000|            N|    20070701|        20070701|        N|20090701|    20090701|
|   00000006|            N|    20180101|        20191231|        N|00000000|    00000000|
|   00000008|            N|    20140101|        20211231|        N|00000000|    00000000|
|   00000011|            S|    20070701|        00000000|        N|00000000|    00000000|
|   00000013|            S|    20090101|        00000000|        N|00000000|    00000000|
+-----------+-------------+------------+----------------+---------+--------+------------+
only showing top 5 rows



In [209]:
df_empresas = df2.withColumn("SIMPLES_MEI_ID",row_number().over(Window.orderBy(monotonically_increasing_id()))-1)

In [210]:
df_empresas.show(5)

22/06/16 02:44:23 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/16 02:44:23 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/16 02:44:58 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/16 02:44:58 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-----------+-------------+------------+----------------+---------+--------+------------+--------------+
|CNPJ_BASICO|OPCAO_SIMPLES|DATA_SIMPLES|SIMPLES_EXCLUSAO|OPCAO_MEI|DATA_MEI|MEI_EXCLUSAO|SIMPLES_MEI_ID|
+-----------+-------------+------------+----------------+---------+--------+------------+--------------+
|   00000000|            N|    20070701|        20070701|        N|20090701|    20090701|             0|
|   00000006|            N|    20180101|        20191231|        N|00000000|    00000000|             1|
|   00000008|            N|    20140101|        20211231|        N|00000000|    00000000|             2|
|   00000011|            S|    20070701|        00000000|        N|00000000|    00000000|             3|
|   00000013|            S|    20090101|        00000000|        N|00000000|    00000000|             4|
+-----------+-------------+------------+----------------+---------+--------+------------+--------------+
only showing top 5 rows



In [211]:
df_empresas.coalesce(1).write \
  .mode('overwrite') \
  .option("header", "True") \
  .csv("gs://neo4j_nunes/dp_socios_dataset/2_CSV_DATASET/ALL_SIMPLES_MEI")

22/06/16 02:45:33 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/16 02:45:33 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/16 02:45:55 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/06/16 02:45:55 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [ ]:
df_1 = spark \
  .read \
  .option ( "inferSchema" , "false" ) \
  .option ( "header" , "false" ) \
  .option("sep", ";") \
  .csv ( "gs://neo4j_nunes/dp_socios_dataset/2_CSV_DATASET/SITUACAO_CADASTRAL" )

In [ ]:
df_1.printSchema()

In [ ]:
df_1.show(5)

In [ ]:
df2 = df_1.selectExpr("_c0 as CODIGO_MUNICIPIO","_c1 as NOME_MUNICIPIO")

In [ ]:
df2.show(5)

In [ ]:
df_empresas = df2.withColumn("SITUACAO_CADASTRAL_ID",row_number().over(Window.orderBy(monotonically_increasing_id()))-1)

In [ ]:
df_empresas.show(5)

In [ ]:
df_empresas.coalesce(1).write \
  .mode('overwrite') \
  .option("header", "True") \
  .csv("gs://neo4j_nunes/dp_socios_dataset/2_CSV_DATASET/ALL_SITUACAO_CADASTRAL")